Exercise 0.1

In [ ]:
# Install and import libraries
import pandas as pd
import duckdb
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset

df = pd.read_csv("salaries.csv")

In [ ]:
#  a) Start with some simple EDA, check summary statistics, info, columns etc

#First rows
df.head()

In [ ]:
# Column names
df.columns



In [ ]:
# Shape

df.shape

In [ ]:
# Info and missing values

df.info()


In [ ]:
# Summary statistics

df.describe()


In [ ]:
 # b) Plot the distribution of the salaries in USD

# Plot histogram

plt.figure()
plt.hist(df["salary_in_usd"], bins=50)
plt.title("Distribution of Salaries in USD")
plt.xlabel("Salary in USD")
plt.ylabel("Number of Job Postings")
plt.show()


In [ ]:
# c) How many job postings are there for each job title? Take the tenth most common job title and plot a bar chart of the number of job postings for that title


# Count job titles

job_counts = df["job_title"].value_counts()
job_counts.head(10)


# Plot bar chart
plt.figure()
job_counts.head(10).plot(kind="bar")
plt.title("Top 10 Most Common Job Titles")
plt.xlabel("Job Title")
plt.ylabel("Number of Job Postings")
plt.show()



In [ ]:
# Get the 10th most common job title

tenth_job = job_counts.index[9]
tenth_job


In [ ]:
# Plot bar chart

plt.figure()
job_counts.head(10).plot(kind="bar")
plt.title("Top 10 Most Common Job Titles")
plt.xlabel("Job Title")
plt.ylabel("Number of Job Postings")
plt.show()


In [ ]:
# d) Create a dataframe that contains the following columns: job_title, experience_level, median_salary_usd, mean_salary_usd, mean_salary_sek. Sort the dataframe by mean_salary_sek.

# Create DuckDB connection
con = duckdb.connect()
con.register("salaries", df)



In [ ]:
# Create aggregated dataframe

salary_stats = con.sql("""
SELECT
    job_title,
    experience_level,
    MEDIAN(salary_in_usd) AS median_salary_usd,
    AVG(salary_in_usd) AS mean_salary_usd,
    AVG(salary_in_usd) * 11 AS mean_salary_sek
FROM salaries
GROUP BY job_title, experience_level
ORDER BY mean_salary_sek DESC
""").df()

# View result

salary_stats.head()


In [ ]:
# f) Create a column with salary in SEK per month and make the distribution of salaries in SEK per month for all job postings

#DVS. Salary in SEK per month and distribution

# Create new columns

df["salary_sek_month"] = (df["salary_in_usd"] * 11) / 12


In [ ]:
# Plot distribution

plt.figure()
plt.hist(df["salary_sek_month"], bins=50)
plt.title("Monthly Salary Distribution in SEK")
plt.xlabel("Salary SEK per Month")
plt.ylabel("Number of Job Postings")
plt.show()


In [ ]:
# g) Now create distributions of monthly SEK based on different experience levels, do you see any trends?

# Dvs. Monthly SEK distributions by experience level

# Loop through experience levels

for level in df["experience_level"].unique():
    subset = df[df["experience_level"] == level]
    plt.figure()
    plt.hist(subset["salary_sek_month"], bins=40)
    plt.title(f"Monthly Salary SEK Distribution for {level}")
    plt.xlabel("Salary SEK per Month")
    plt.ylabel("Count")
    plt.show()


Trend discussion

- Higher experience levels have higher medians

- Senior roles show wider salary spread

- Entry level salaries are more concentrated

In [ ]:
# h) Find median monthly swedish salary for more common job_titles. Think what common may mean here.

# Dvs. Median monthly Swedish salary for common job titles

# Define common job titles (for least 50 postings)

common_jobs = df["job_title"].value_counts()
common_jobs = common_jobs[common_jobs >= 50].index


# Filter and calculate median

sweden_df = df[df["company_location"] == "SE"]
sweden_common = sweden_df[sweden_df["job_title"].isin(common_jobs)]

median_sweden = (
    sweden_common
    .groupby("job_title")["salary_sek_month"]
    .median()
    .sort_values(ascending=False)
)


# View result

median_sweden


In [ ]:
# i) Join the country_codes.csv to the dataset get the actual country names. Find the median monthly salary in SEK for each country and the number of job postings.

# Dvs. Join country codes and analyze


# Load country codes

countries = pd.read_csv("country_codes.csv")


In [ ]:
# Join datasets

df_joined = df.merge(
    countries,
    left_on="company_location",
    right_on="country_code",
    how="left"
)


In [ ]:
# Median salary and postings per country

country_stats = (
    df_joined
    .groupby("country_name")
    .agg(
        median_salary_sek_month=("salary_sek_month", "median"),
        job_postings=("job_title", "count")
    )
    .sort_values("median_salary_sek_month", ascending=False)
)


In [ ]:
# View result

country_stats
